In [1]:
import netCDF4 as nc
from matplotlib import pyplot as plt
plt.ion()
import numpy as np
from netcdftime import utime
import os
import rasterio
import shutil

In [2]:
YEAR='2014'

In [3]:
fname = nc.Dataset('../data/icecharts-{}/merged-icecharts-{}.nc'.format(YEAR, YEAR))

In [4]:
fastice_dst = fname['ice_concentration'][:].astype(np.bool)

In [5]:
tsmp = fname['time']
cdftime = utime(tsmp.units)
timestamps = cdftime.num2date(tsmp[:])

In [9]:
count= np.zeros(fastice_dst.shape[1::])
mask = np.ones(count.shape, dtype=np.bool)
switch = np.ones(count.shape)
for i in range(fastice_dst.shape[0]):
    switch *= fastice_dst[i]
    count += switch
count = np.where(count<=1, 0, count)    

In [10]:
#write rle to file
import shutil
template_tif = '/media/DataLocal/Projects/FjordIcePersistency/templates/svalbard_template.tif'
ofile_tif = 'svalbard_fastice_persistency_{}.tif'.format(YEAR)
shutil.copy(template_tif, ofile_tif)

In [11]:
template = rasterio.open(template_tif, 'r')
meta = template.meta.copy()

with rasterio.open(ofile_tif, 'w', **meta) as dst:
        dst.write((count/5.*7.).astype(rasterio.uint8), 1)

/home/mikhail/.virtualenvs/fjordice/local/lib/python2.7/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [12]:
template.close()